# Connect to the API

In [4]:
import pprint as pp
import requests
from opensearchpy import OpenSearch
from opensearchpy import helpers

host = 'api.novasearch.org'
port = 443
user = 'user201' # Add your user name here.
password = 'Lrr1531' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

# Delete an index

In [8]:
response = client.indices.delete(
        index = user
    )
print(response)

{'acknowledged': True}


# Index creation

In [9]:

index_body = {
   "settings":{
      "index":{
         "number_of_replicas":0,
         "number_of_shards":4,
         "refresh_interval":"-1",
         "knn":"true"
      }
   },
   "mappings":{
       "dynamic":      "strict",
       "properties":{
         "doc_id":{
            "type":"keyword"
         },
         "tags":{
            "type":"keyword"
         },
         "contents":{
            "type":"text",
            "analyzer":"standard",
            "similarity":"BM25"
         }
      }
   }
}

if client.indices.exists(index=index_name):
    print("Index already existed. Nothing to be done.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'user201'}


# Test

In [70]:
import json as json

with open("./recipes/recipes_data.json", "r") as read_file:
    data = json.load(read_file)

j = len(data)
str(j)

for key, recipe in data.items():
    ingredients = recipe['ingredients']
    for ingredient in ingredients:
        print(ingredient['displayText'])

½ teaspoon of red pepper flakes (optional)
¼ cup of freshly grated Parmesan cheese
4 ounces of part-skim mozzarella, sliced
4 ounces of fresh mozzarella, sliced
1 ½ cups of marinara sauce
4 boneless, skinless chicken breasts (1 ½ to 1 ¾ pounds total)
1 ½ cups of Italian-seasoned breadcrumbs
1 tablespoon of water
3 large eggs
1 teaspoon of freshly ground black pepper, divided
2 teaspoons of kosher salt, divided, plus more for seasoning
1 cup of all-purpose flour
1 ¼ to 2 cups of olive oil (not extra-virgin)
¼ to ½ cup of extra-virgin olive oil
½ teaspoon of kosher salt
1 to 2 cloves of garlic
½ cup of grated Parmesan cheese, or any other hard cheese
½ cup of toasted pine nuts, or any other nut
5 to 6 ounces of fresh basil leaves (2 big bunches or about 6 cups gently packed), or any other green, divided
1 ½ cups of hot water (hot tap water is fine)
½ teaspoon of kosher salt
2 cups of masa harina
½ cup of mayonnaise
½ cup of Mexican crema or sour cream
1 medium lime, cut into wedges, for 